# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? 
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [1]:
import pandas

df = pandas.read_csv("./data/airbnb_hw.csv")

print(df['Price'].head())

df['Price'] = (
    df['Price']
    .astype(str)
    .str.replace(',', '', regex=False)
    .str.replace('$', '', regex=False)
    .str.strip()
)
df['Price'] = pandas.to_numeric(df['Price'], errors="coerce")

missing_count = df["Price"].isna().sum()
print("Missing values in Price:", missing_count)
with_commas = df[df["Price"].astype(str).str.contains(",")]
print("Rows with commas in Price formatting:\n", with_commas)
df["Price"] = df["Price"].replace(",", "", regex=True)
df["Price"] = pandas.to_numeric(df["Price"], errors="coerce")
missing_after = df["Price"].isna().sum()
print("Missing values in Price after cleaning:", missing_after)

# For this question, I cleaned the 'Price' column by removing dollar signs ("$") and commas (",") 
# so the values could be converted into a numeric format. I also stripped out any extra spaces. 
# After converting the column to numbers, I checked the number of missing values before and after cleaning 
# to make sure the transformation worked correctly. 


0    145
1     37
2     28
3    199
4    549
Name: Price, dtype: object
Missing values in Price: 0
Rows with commas in Price formatting:
 Empty DataFrame
Columns: [Host Id, Host Since, Name, Neighbourhood , Property Type, Review Scores Rating (bin), Room Type, Zipcode, Beds, Number of Records, Number Of Reviews, Price, Review Scores Rating]
Index: []
Missing values in Price after cleaning: 0


In [2]:
import pandas

df = pandas.read_csv("./data/mn_police_use_of_force.csv")

df['subject_injury'] = (
    df['subject_injury']
    .astype(str)
    .str.strip()
    .str.lower()
)

df['subject_injury'] = df['subject_injury'].replace({
    'nan': 'Missing',
    'yes': 'Yes',
    'no': 'No'
})

print(df['subject_injury'].value_counts(normalize=True))

# cross-tab with force_type
print(pandas.crosstab(df['subject_injury'], df['force_type']))

# I cleaned the 'subject_injury' column by converting all values to strings, 
# stripping extra spaces, and making everything lowercase for consistency. 
# Then I replaced 'nan' with 'Missing' to clearly mark missing data, 
# and standardized the responses so that 'yes' became 'Yes' and 'no' became 'No'.


subject_injury
Missing    0.761934
Yes        0.126190
No         0.111876
Name: proportion, dtype: float64
force_type      Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury                                                    
Missing             2          7051               1421        0   
No                  0          1093                131        2   
Yes                 2          1286                 41        0   

force_type      Gun Point Display  Improvised Weapon  Less Lethal  \
subject_injury                                                      
Missing                        27                 74           87   
No                             33                 34            0   
Yes                            44                 40            0   

force_type      Less Lethal Projectile  Maximal Restraint Technique  \
subject_injury                                                        
Missing                              0                          170 

In [3]:
import pandas
import numpy as np

df = pandas.read_parquet("./data/justice_data.parquet")

df["WhetherDefendantWasReleasedPretrial"] = df["WhetherDefendantWasReleasedPretrial"].replace(
    ["", "NA"], np.nan
)

print(df["WhetherDefendantWasReleasedPretrial"].head(10))

# I cleaned the column "WhetherDefendantWasReleasedPretrial" by replacing empty strings ("") 
# and "NA" values with np.nan so that the missing data is properly recognized in the dataframe. 
# This helps make the dataset more consistent and ready for analysis. 


0    9
1    0
2    0
3    0
4    1
5    0
6    1
7    1
8    0
9    1
Name: WhetherDefendantWasReleasedPretrial, dtype: int64


In [4]:
import pandas as pd
import numpy as np

df = pandas.read_parquet("./data/justice_data.parquet") 

df["CleanedImposedSentence"] = np.where(
    (df["ImposedSentenceAllChargeInContactEvent"].isna()) &
    (df["SentenceTypeAllChargesAtConvictionInContactEvent"].isin(["No Sentence", "Acquitted"])),
    0,
    df["ImposedSentenceAllChargeInContactEvent"]
)

def clean_sentence(row):
    if pd.isna(row["ImposedSentenceAllChargeInContactEvent"]):
        if row["SentenceTypeAllChargesAtConvictionInContactEvent"] in ["No Sentence", "Acquitted"]:
            return 0
    return row["ImposedSentenceAllChargeInContactEvent"]

df["CleanedImposedSentence"] = df.apply(clean_sentence, axis=1)

print(df[["SentenceTypeAllChargesAtConvictionInContactEvent",
          "ImposedSentenceAllChargeInContactEvent",
          "CleanedImposedSentence"]].head(15))

# I cleaned the "ImposedSentenceAllChargeInContactEvent" column by replacing missing values with 0 
# when the "SentenceTypeAllChargesAtConvictionInContactEvent" was either "No Sentence" or "Acquitted." 
# Otherwise, I kept the original imposed sentence values. I saved the cleaned results in a new column 
# called "CleanedImposedSentence."

    SentenceTypeAllChargesAtConvictionInContactEvent  \
0                                                  9   
1                                                  0   
2                                                  1   
3                                                  1   
4                                                  4   
5                                                  0   
6                                                  1   
7                                                  0   
8                                                  1   
9                                                  1   
10                                                 0   
11                                                 4   
12                                                 4   
13                                                 1   
14                                                 1   

   ImposedSentenceAllChargeInContactEvent CleanedImposedSentence  
0                                   

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

In [5]:
!pip install xlrd>=2.0.1
!pip install openpyxl

import pandas as pd
import numpy as np

df = pd.read_excel("data/GSAF5.xls", engine="xlrd")  # Explicitly specify the engine

print("Column names in the DataFrame:", df.columns.tolist())

df = df.dropna(axis=1, how="all")

df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
print("Year range:", df["Year"].min(), "to", df["Year"].max())

df_recent = df[df["Year"] >= 1940]
attacks_per_year = df_recent.groupby("Year").size()
print("\nNumber of attacks per year since 1940 (last 10 years shown):")
print(attacks_per_year.tail(10))

df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
print("\nAge summary of victims:")
print(df["Age"].describe())

sex_column = "Sex " if "Sex " in df.columns else "Sex"
df[sex_column] = df[sex_column].str.strip().str.upper()
prop_male = (df[sex_column] == "M").mean()
print("\nProportion of victims that are male:", prop_male)

df["Type"] = df["Type"].str.strip().str.capitalize()
df.loc[~df["Type"].isin(["Provoked", "Unprovoked"]), "Type"] = "Unknown"
prop_unprovoked = (df["Type"] == "Unprovoked").mean()
print("Proportion of attacks that are unprovoked:", prop_unprovoked)

df["Fatal Y/N"] = df["Fatal Y/N"].str.strip().str.upper()
df.loc[~df["Fatal Y/N"].isin(["Y", "N"]), "Fatal Y/N"] = "Unknown"

print("\nFatal by gender:")
print(pd.crosstab(df["Sex"], df["Fatal Y/N"], normalize="index"))

print("\nFatal by type:")
print(pd.crosstab(df["Type"], df["Fatal Y/N"], normalize="index"))

print("\nUnprovoked vs Provoked by gender:")
print(pd.crosstab(df["Sex"], df["Type"], normalize="index"))

df["Species "] = df["Species "].str.lower()
species_split = df["Species "].dropna().str.split()
is_white_shark = species_split.apply(lambda x: "white" in x)
prop_white = is_white_shark.mean()
print("\nProportion of attacks by white sharks:", prop_white)

# I cleaned the shark attack dataset by dropping empty columns and fixing the Year and Age columns 
# so they are numeric and easier to analyze. I standardized the Sex values by stripping spaces and 
# making them uppercase, then calculated the proportion of male victims. I also cleaned the Type 
# column so it only had "Provoked," "Unprovoked," or "Unknown," and fixed the Fatal Y/N column 
# to only include "Y," "N," or "Unknown." Finally, I lowercased the Species column and used str.split() 
# to check what proportion of attacks were from white sharks.

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Column names in the DataFrame: ['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ', 'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1', 'original order', 'Unnamed: 21', 'Unnamed: 22']
Year range: 0.0 to 2026.0

Number of attacks per year since 1940 (last 10 years shown):
Year
2017.0    141
2018.0    124
2019.0    114
2020.0    101
2021.0    110
2022.0     98
2023.0    109
2024.0     52
2025.0     48
2026.0      1
dtype: int64

Age summary of victims:
count    3903.000000
mean       28.154497
std        14.649413
min         1.000000
25%        17.000000
50%        24.000000
75%        37.000000
max        87.000000
Name: Age, dtype: float64

Proportion of victims that are male: 0.8026128940641863
Proportion of attacks that are unprovoked: 0.7380005680204488

Fatal by ge

In [1]:
import os
os.getcwd()


'/home/8f4def31-7c4d-4735-aa8c-7f63997add49'